# Домашнее задание 2 (продвинутое)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,5)

# Для кириллицы на графиках
font = {'family': 'Verdana',
        'weight': 'normal'}
plt.rc('font', **font)

try:
    from ipywidgets import interact, IntSlider, fixed, FloatSlider
except ImportError:
    print(u'Так надо')

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Задание

Ваша задада - научиться предсказывать рейтинг (тональность) отзывов к фильмам Amazon


Загрузите текстовые данные [отсюда](https://archive.ics.uci.edu/ml/machine-learning-databases/00331/). Архив должен содержать 3 файла с положительными и отрицательными отзывами с ресурсов
* imdb.com
* amazon.com
* yelp.com

Формат файла следующий:
<отзыв>\t<метка>\n


### Задача
1. Загрузите тексты и метки классов в разные переменные
2. Мера качества классификации - accuracy
3. Обучите линейный SVM (без подбора гиперпараметров). Тексты представляются в виде мешка слов
4. Выведите наиболее значимые слова из текста
5. С помощью кросс-валидации и валидационных кривых исследуйте, как различные комбинции параметров влияют на качество
6. Сравните с другими линейными методами классификации
7. Визуализируйте обучающую выборку и предсказания классификатора с помощью PCA и (или) t-SNE

# Подсказки (бейслайн)

In [2]:
ls ./sentiment\ labelled\ sentences

amazon_cells_labelled.txt  readme.txt
imdb_labelled.txt          yelp_labelled.txt


## Загрузите тексты и метки классов в разные переменные

In [3]:
'''
This dataset was created for the Paper 'From Group to Individual Labels using Deep Features', Kotzias et. al,. KDD 2015
Please cite the paper if you want to use it :)

It contains sentences labelled with positive or negative sentiment, extracted from reviews of products, movies, and restaurants

=======
Format:
=======
sentence \t score \n


=======
Details:
=======
Score is either 1 (for positive) or 0 (for negative)	
The sentences come from three different websites/fields:

imdb.com
amazon.com
yelp.com

For each website, there exist 500 positive and 500 negative sentences. Those were selected randomly for larger datasets of reviews. 
We attempted to select sentences that have a clearly positive or negative connotaton, the goal was for no neutral sentences to be selected.



For the full datasets look:

imdb: Maas et. al., 2011 'Learning word vectors for sentiment analysis'
amazon: McAuley et. al., 2013 'Hidden factors and hidden topics: Understanding rating dimensions with review text'
yelp: Yelp dataset challenge http://www.yelp.com/dataset_challenge
'''

files = ['amazon_cells_labelled.txt', 'imdb_labelled.txt', 'yelp_labelled.txt']
texts, labels = [], []

# Сливаем все отзывы в texts
# Сливаем всю разметку по тональности в labels

for file in files:
    with open('./sentiment labelled sentences/' + file) as f:
        data = f.readlines()
        data = [i.strip('\n').split('\t') for i in data]
        for i in data:
            texts.append(i[0])
            labels.append(i[1])

# Разобъем на тестовую и валидационную выборку

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, train_size=0.8)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Обучите линейный SVM (без подбора гиперпараметров). Тексты представляются в виде мешка слов

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from mlxtend.preprocessing import DenseTransformer
from sklearn.svm import SVC


model = Pipeline(
[
    ('vect', TfidfVectorizer()), # Позволяет преобразовать текст в набор признаков, в значении каждого из которых стоит нормированная на число документов частота соответствующего слова
    ('dense', DenseTransformer()), # TfidfVectorizer возвращает SparseMatrix. Нам нужно преобразовать её в Dense, чтобы передать классификатору
    ('svm', SVC(kernel='linear')) # Функция потерь встроена
])

In [6]:
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

# Мера качества классификации - accuracy

In [7]:
# Оценим по заявленной метрике качество
# ФУНКЦИОНАЛ КАЧЕСТВА
from sklearn.metrics import accuracy_score
model_test_predictions = model.predict(X_test)
accuracy_score(model_test_predictions, y_test)

0.8216666666666667

# Выведите наиболее значимые слова из текста

In [8]:
# Посмотрим что вообще есть
for i in model.get_params().keys():
    print(i, dir(i), '\n')

memory ['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketrans', 'partition', 'replace', 'rfind', 'rindex', 'rjust', 'rpartition', 'rsplit', 'rstrip', 'split', 'splitlines', 'startswith', 'strip', 'swapcase', 'title', 'translate', 'upper', 'zfill'] 

steps ['__add__', '__class__', '__contains__', '__delattr__', '__dir_

In [9]:
# объедим названия признаков и их важность в один список методом zip, 
# отсортируем в порядке возрастания коэффициентов до 10
feature_importances = list(sorted(zip(texts, model.steps[2][1].coef_.squeeze()), key=lambda x: -abs(x[1]))) 
pd.DataFrame(feature_importances, columns=['feature_name', 'feature_importance'])[:10]

,feature_name,feature_importance
0,Google mediocre and I imagine Smashburger will...,-4.702725
1,I am so tired of clichés that is just lazy wri...,3.775212
2,"A very good film indeed, about great and uncon...",2.961202
3,I was able to do voice dialing in the car with...,-2.724569
4,I came over from Verizon because cingulair has...,2.685586
5,"Our waiter was very attentive, friendly, and i...",2.398248
6,An hour and a half I wish I could bring back.,-2.391030
7,But I don't like it.,-2.228847
8,How can that be?The audio quality is poor.,2.127265
9,The buffet is small and all the food they offe...,2.123945


# С помощью кросс-валидации и валидационных кривых исследуйте, как различные комбинции параметров влияют на качество

In [10]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, labels, train_size=0.85, random_state=13)

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
from tqdm import tqdm

results = {}

# TfidfVectorizer
norms = ['l1', 'l2'] # norm=
use_idf=[True, False] #use_idf=

# SVC
kernels = ['linear', 'poly', 'rbf', 'sigmoid'] #kernel=
decisions = ['ovo', 'ovr'] #decision_function_shape=

for n in norms:
    for u in use_idf:
        for k in kernels:
            for d in tqdm(decisions):
                model = Pipeline(
                [
                    ('vect', TfidfVectorizer(norm=n, use_idf=u)), # Позволяет преобразовать текст в набор признаков, в значении каждого из которых стоит нормированная на число документов частота соответствующего слова
                    ('dense', DenseTransformer()), # TfidfVectorizer возвращает SparseMatrix. Нам нужно преобразовать её в Dense, чтобы передать классификатору
                    ('svm', SVC(kernel=k, decision_function_shape=d))
                ])
                model.fit(X_train, y_train)
                model_test_predictions = model.predict(X_test)
                res = accuracy_score(model_test_predictions, y_test)
                results.update({(n, u, k, d): res})

100%|██████████| 2/2 [01:52<00:00, 56.34s/it]


# Сравните с другими линейными методами классификации

In [12]:
pd.DataFrame.from_dict(results, orient='index')

,0
"(l1, True, linear, ovo)",0.768889
"(l1, True, linear, ovr)",0.768889
"(l1, True, poly, ovo)",0.497778
"(l1, True, poly, ovr)",0.497778
"(l1, True, rbf, ovo)",0.497778
"(l1, True, rbf, ovr)",0.497778
"(l1, True, sigmoid, ovo)",0.497778
"(l1, True, sigmoid, ovr)",0.497778
"(l1, False, linear, ovo)",0.713333
"(l1, False, linear, ovr)",0.713333
